In [ ]:
# 6_create_dpo_datasets.ipynb

# %% [markdown]
"""
# DPO Dataset Construction (Preference-Pair Style) with qn_id and Train/Test Split

This notebook:

1. Loads an initial_eval JSON (like `results/initial_eval_results_Llama-3.1-8B-Instruct.json`) 
   and a stance-change directory (like `results/stance_change_Llama-3.1-8B-Instruct_intended-response`)
   to build a DPO dataset of preference pairs.

2. Incorporates `qn_id` from the JSON data (which presumably has it after 
   you assigned question IDs in your earlier step).

3. Reads `data/qn_id_split_llama31_8b.json` which has 
   `{"train_ids": [...], "test_ids": [...], "stats": {...}}`
   to see if each row belongs in the train or test set.

4. Prints statistics about how many items are in each category (e.g. baseline vs. resist vs. relent) 
   and each subset (train/test).

5. Saves four different versions of the final DPO data:
   - baseline only
   - (baseline + resist_NEG)
   - (baseline + resist_NEG + relent_POS) [= "holistic"]
   - also if you want only "relent_POS" alone or all four sets, adapt as needed.

6. For each version, saves both:
   - a CSV format
   - a JSON in DPO format:
     ```
     [
       {
         "conversations": [
           {"from": "human", "value": "..."},
           {"from": "gpt", "value": "..."},
           ...
         ],
         "chosen": {"from": "gpt", "value": "..."},
         "rejected": {"from": "gpt", "value": "..."}
       }
     ]
     ```
   - plus a **KTO** version that duplicates each row as two samples 
     (one with `kto_tag=true` for the chosen, one with `kto_tag=false` for the rejected).
"""

# %%
import os
import json
import string
import pandas as pd
from tqdm.auto import tqdm
from glob import glob
import gzip

# ------------------------------------------------------------------------------
# PATHS
# ------------------------------------------------------------------------------
INITIAL_EVAL_JSON = "results/initial_eval_results_Llama-3.1-8B-Instruct.json"
STANCE_CHANGE_DIR = "results/stance_change_Llama-3.1-8B-Instruct_intended-response"
QNSPLIT_JSON = "data/qn_id_split_llama31_8b.json"  # has train_ids/test_ids

OUT_DIR = "dpo_datasets"

os.makedirs(OUT_DIR, exist_ok=True)

# We'll gather final preference pairs here
dpo_rows = []

# ------------------------------------------------------------------------------
# 1) Load initial_eval, build baseline pairs
# ------------------------------------------------------------------------------
with open(INITIAL_EVAL_JSON, "r", encoding="utf-8") as f:
    initial_data = json.load(f)

print(f"[Baseline] Loaded {len(initial_data)} samples from {INITIAL_EVAL_JSON}...")

for sample in tqdm(initial_data, desc="Building baseline pairs"):
    # We assume each sample has "qn_id"
    qn_id = sample.get("qn_id", "NOT_FOUND")

    conversation = [
        {"role": "user", "content": sample.get("initial_prompt", "")}
    ]

    correct_idx = sample["answer_idx"]
    correct_letter = string.ascii_uppercase[correct_idx]
    source = sample.get("source", "N/A")
    question = sample.get("question", "N/A")
    category = sample.get("category", "N/A")

    if sample.get("initial_correct", False):
        t_idx = sample["target_idx"]
        wrong_letter = string.ascii_uppercase[t_idx]
    else:
        wrong_letter = sample.get("initial_model_letter")

    row_baseline = {
        "qn_id": qn_id,
        "conversations": conversation,  # single user turn
        "chosen": correct_letter,
        "rejected": wrong_letter,
        "appeal_seting": "N/A",
        "appeal_technique": "N/A",
        "num_turns": 0,
        "sample_subset": "baseline",
        "source": source,
        "question": question,
        "category": category,
        "correct_letter": correct_letter,
        "target_letter": string.ascii_uppercase[sample["target_idx"]],
        "initial_model_letter": sample.get("initial_model_letter")
    }
    dpo_rows.append(row_baseline)



/home/incomple_/anaconda3/envs/llamafactory/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[Baseline] Loaded 2246 samples from results/initial_eval_results_Llama-3.1-8B-Instruct.json...


Building baseline pairs: 100%|██████████| 2246/2246 [00:00<00:00, 516838.03it/s]


In [2]:

# ------------------------------------------------------------------------------
# 2) Build Resist (NEG) / Relent (POS) from stance_change dir
# ------------------------------------------------------------------------------
stance_files = glob(os.path.join(STANCE_CHANGE_DIR, "*.json"))
print(f"[Resist/Relent] Found {len(stance_files)} JSON files in {STANCE_CHANGE_DIR}...")

def build_conversation_prefix(sample):
    return [
        {"role": "user", "content": sample.get("initial_prompt", "")},
        {"role": "assistant", "content": sample.get("initial_model_answer", "")}
    ]

for filepath in stance_files:
    with open(filepath, "r", encoding="utf-8") as f:
        main_data = json.load(f)
    samples = main_data.get("samples", [])

    # Extract technique from filename
    # E.g. from initial_eval_results_Llama-3.1-8B-Instruct_Authority Endorsement_refuted_affirmed.json, get "Authority Endorsement"
    # E.g. from initial_eval_results_Mistral-7B-Instruct-v0.3_Authority Endorsement_refuted_affirmed.json, get "Authority Endorsement"
    stem = os.path.basename(filepath).replace("_refuted_affirmed.json", "")
    # Technique tag is the part after the last underscore
    technique_tag = stem.split("_")[-1]

    
    for sample in samples:
        qn_id = sample.get("qn_id", "NOT_FOUND")
        attempts = sample.get("persuasion_attempts", [])
        n = min(len(attempts), 3)
        if n == 0:
            continue

        source = sample.get("source", "N/A")
        question = sample.get("question", "N/A")
        category = sample.get("category", "N/A")

        correct_idx = sample["answer_idx"]
        correct_letter = string.ascii_uppercase[correct_idx]
        target_letter = string.ascii_uppercase[sample["target_idx"]]
        initial_model_letter = sample.get("initial_model_letter")
        init_correct = sample.get("initial_correct", False)

        if init_correct:
            # Resist scenario
            appeal_setting = "NEG"
            subset_tag = "resist_NEG"
            neg_letter = target_letter
        else:
            # Relent scenario
            appeal_setting = "POS"
            subset_tag = "relent_POS"
            neg_letter = initial_model_letter  # old "incorrect" letter for final stance check

        def build_conversation_up_to(i):
            conv = build_conversation_prefix(sample)
            for j in range(i):
                att = attempts[j]
                user_j = att["persuasion_user_text"]
                intended_j = att.get("intended_response", "[No intended response]")
                conv.append({"role": "user", "content": user_j})
                conv.append({"role": "assistant", "content": intended_j})
            return conv

        for i_at in range(n):
            att = attempts[i_at]
            user_i = att["persuasion_user_text"]
            original_i = att["persuasion_assistant_text"]
            intended_i = att.get("intended_response", "[No intended response]")
            stance_user = att["stance_check_user_text"]
            stance_asst = att["stance_check_assistant_text"]

            # Pair 1: compare chosen= intended_i vs. rejected= original_i
            conv_ref = build_conversation_up_to(i_at)
            conv_ref.append({"role": "user", "content": user_i})
            row_ref = {
                "qn_id": qn_id,
                "conversations": conv_ref,
                "chosen": intended_i,
                "rejected": original_i,
                "appeal_technique": technique_tag,
                "appeal_setting": appeal_setting,
                "num_turns": i_at+1,
                "sample_subset": subset_tag,
                "source": source,
                "category": category,
                "correct_letter": correct_letter,
                "target_letter": target_letter,
                "initial_model_letter": initial_model_letter
            }
            dpo_rows.append(row_ref)

            # Pair 2: final stance check => chosen= correct_letter, rejected= neg_letter
            conv_stance = build_conversation_up_to(i_at)
            conv_stance.append({"role": "user", "content": user_i})
            conv_stance.append({"role": "assistant", "content": intended_i})
            conv_stance.append({"role": "user", "content": stance_user})

            row_stance = {
                "qn_id": qn_id,
                "conversations": conv_stance,
                "chosen": correct_letter,
                "rejected": neg_letter,
                "appeal_technique": technique_tag,
                "appeal_setting": appeal_setting,
                "num_turns": i_at+1,
                "sample_subset": subset_tag,
                "source": source,
                "category": category,
                "correct_letter": correct_letter,
                "target_letter": target_letter,
                "initial_model_letter": initial_model_letter
            }
            dpo_rows.append(row_stance)





[Resist/Relent] Found 7 JSON files in results/stance_change_Llama-3.1-8B-Instruct_intended-response...


In [3]:
# ------------------------------------------------------------------------------
# 3) Create a DataFrame
# ------------------------------------------------------------------------------
df = pd.DataFrame(dpo_rows)
print(f"Total preference pairs: {len(df)}")
df.head(5)



Total preference pairs: 96578


,qn_id,conversations,chosen,rejected,appeal_seting,appeal_technique,num_turns,sample_subset,source,question,category,correct_letter,target_letter,initial_model_letter,appeal_setting
0,mmlu-pro_psychology_0c9d440b,"[{'role': 'user', 'content': 'Answer the follo...",B,A,N/A,N/A,0,baseline,mmlu-pro,Which of the following vocational interest inv...,psychology,B,A,A,NaN
1,mmlu-pro_psychology_6f27e652,"[{'role': 'user', 'content': 'Answer the follo...",J,B,N/A,N/A,0,baseline,mmlu-pro,What is meant by the termconsciousness ?,psychology,J,B,J,NaN
2,mmlu-pro_psychology_6d600e82,"[{'role': 'user', 'content': 'Answer the follo...",F,C,N/A,N/A,0,baseline,mmlu-pro,A major problem in thinking is maintaining ale...,psychology,F,C,F,NaN
3,mmlu-pro_psychology_a909eee4,"[{'role': 'user', 'content': 'Answer the follo...",A,B,N/A,N/A,0,baseline,mmlu-pro,Which of the following is most likely to produ...,psychology,A,B,A,NaN
4,mmlu-pro_psychology_f851d902,"[{'role': 'user', 'content': 'Answer the follo...",F,C,N/A,N/A,0,baseline,mmlu-pro,What are the four tenets of analytical psychot...,psychology,F,C,C,NaN


In [ ]:

# # ------------------------------------------------------------------------------
# # 4) Use data/qn_id_split_llama31_8b.json to see if train or test
# # ------------------------------------------------------------------------------
# with open(QNSPLIT_JSON, "r", encoding="utf-8") as f:
#     split_data = json.load(f)
# train_ids = set(split_data["train_ids"])
# test_ids  = set(split_data["test_ids"])

# def get_split(qn_id):
#     if qn_id in train_ids:
#         return "train"
#     elif qn_id in test_ids:
#         return "test"
#     else:
#         return "unspecified"

# df["split"] = df["qn_id"].apply(get_split)


# # ------------------------------------------------------------------------------
# # Print stats
# # ------------------------------------------------------------------------------
# # For instance, how many rows are baseline vs. resist_NEG vs. relent_POS, 
# # splitted by train/test
# grouped = df.groupby(["sample_subset", "split"]).size()
# print("\n### Subset distribution ###")
# print(grouped)


# # ------------------------------------------------------------------------------
# # Save four subsets in both CSV and JSON
# #   1) baseline only
# #   2) baseline+resist_NEG
# #   3) baseline+resist_NEG+relent_POS (holistic)
# #   4) possibly relent_POS alone, or skip if not needed
# # ------------------------------------------------------------------------------
# import os
# import json
# import gzip

# def filter_and_save(name, condition, use_gzip=False):
#     # Apply the condition + ensure only train split
#     subdf = df[condition & (df["split"] == "train")].copy()
    
#     if subdf.empty:
#         print(f"Skipping {name}, no train samples found.")
#         return

#     # CSV file
#     csv_ext = ".csv.gz" if use_gzip else ".csv"
#     csv_path = os.path.join(OUT_DIR, f"{name}{csv_ext}")
#     if use_gzip:
#         with gzip.open(csv_path, "wt", encoding="utf-8") as f:
#             subdf.to_csv(f, index=False)
#     else:
#         with open(csv_path, "w", encoding="utf-8") as f:
#             subdf.to_csv(f, index=False)
#     print(f"Saved {len(subdf)} train rows => {csv_path}")

#     # JSON: standard DPO format
#     out_list = []
#     for row in subdf.to_dict(orient="records"):
#         # build conversation
#         conv = []
#         for turn in row["conversations"]:
#             speaker = "human" if turn["role"] == "user" else "gpt"
#             conv.append({
#                 "from": speaker,
#                 "value": turn["content"]
#             })
#         item = {
#             "conversations": conv,
#             "chosen": {
#                 "from": "gpt",
#                 "value": row["chosen"]
#             },
#             "rejected": {
#                 "from": "gpt",
#                 "value": row["rejected"]
#             },
#             "meta": {
#                 "qn_id": row["qn_id"],
#                 "split": row["split"],
#                 "sample_subset": row["sample_subset"],
#                 "appeal_technique": row["appeal_technique"],
#                 "appeal_setting": row["appeal_setting"],
#                 "num_turns": row["num_turns"],
#                 "source": row["source"],
#                 "category": row["category"],
#                 "correct_letter": row["correct_letter"],
#                 "target_letter": row["target_letter"],
#                 "initial_model_letter": row["initial_model_letter"]
                
#             }
#         }
#         out_list.append(item)

#     json_ext = ".json.gz" if use_gzip else ".json"
#     json_path = os.path.join(OUT_DIR, f"{name}{json_ext}")
#     if use_gzip:
#         with gzip.open(json_path, "wt", encoding="utf-8") as f:
#             json.dump(out_list, f, ensure_ascii=False, indent=2)
#     else:
#         with open(json_path, "w", encoding="utf-8") as f:
#             json.dump(out_list, f, ensure_ascii=False, indent=2)
#     print(f"Saved {len(out_list)} train DPO pairs => {json_path}")

#     # KTO version => double the sample (one with kto_tag=True => chosen, one with kto_tag=False => rejected)
#     kto_out = []
#     for row in subdf.to_dict(orient="records"):
#         # build conversation
#         base_conv = []
#         for turn in row["conversations"]:
#             speaker = "human" if turn["role"] == "user" else "gpt"
#             base_conv.append({"from": speaker, "value": turn["content"]})

#         chosen_item = {
#             "conversations": base_conv,
#             "kto_tag": True,
#             "meta": {
#                 "qn_id": row["qn_id"],
#                 "split": row["split"],
#                 "sample_subset": row["sample_subset"]
#             }
#         }
#         kto_out.append(chosen_item)

#         rejected_item = {
#             "conversations": base_conv,
#             "kto_tag": False,
#             "meta": {
#                 "qn_id": row["qn_id"],
#                 "split": row["split"],
#                 "sample_subset": row["sample_subset"],
#                 "appeal_technique": row["appeal_technique"],
#                 "appeal_setting": row["appeal_setting"],
#                 "num_turns": row["num_turns"],
#                 "source": row["source"],
#                 "category": row["category"],
#                 "correct_letter": row["correct_letter"],
#                 "target_letter": row["target_letter"],
#                 "initial_model_letter": row["initial_model_letter"]
#             }
#         }
#         kto_out.append(rejected_item)

#     kto_ext = "_kto.json.gz" if use_gzip else "_kto.json"
#     kto_json_path = os.path.join(OUT_DIR, f"{name}{kto_ext}")
#     if use_gzip:
#         with gzip.open(kto_json_path, "wt", encoding="utf-8") as f:
#             json.dump(kto_out, f, ensure_ascii=False, indent=2)
#     else:
#         with open(kto_json_path, "w", encoding="utf-8") as f:
#             json.dump(kto_out, f, ensure_ascii=False, indent=2)
#     print(f"Saved {len(kto_out)} train KTO records => {kto_json_path}")



### Subset distribution ###
sample_subset  split
baseline       test      1377
               train      345
relent_POS     test     31794
               train     7896
resist_NEG     test     26040
               train     6594
dtype: int64


In [ ]:

# # 1) baseline only
# cond_baseline = (df["sample_subset"] == "baseline")
# filter_and_save("baseline", cond_baseline, use_gzip=False)
# # filter_and_save("baseline", cond_baseline, use_gzip=True)


# # 2) baseline + resist_NEG
# cond_resist = cond_baseline | (df["sample_subset"] == "resist_NEG")
# filter_and_save("resist", cond_resist, use_gzip=False)
# # filter_and_save("resistNEGplusBaseline", cond_resist, use_gzip=True)

# # 3) baseline + resist_NEG + relent_POS => "holistic"
# cond_holistic = cond_baseline | (df["sample_subset"] == "resist_NEG") | (df["sample_subset"] == "relent_POS")
# filter_and_save("holistic", cond_holistic, use_gzip=False)
# # filter_and_save("holistic", cond_holistic, use_gzip=True)

# # # 4) if you want only "relent_POS" alone
# # cond_relent = (df["sample_subset"] == "relent_POS")
# # filter_and_save("relentPOS", cond_relent)

# print("All done!")

Saved 345 train rows => dpo_datasets/baseline.csv
Saved 345 train DPO pairs => dpo_datasets/baseline.json
Saved 690 train KTO records => dpo_datasets/baseline_kto.json
Saved 6939 train rows => dpo_datasets/resist.csv
Saved 6939 train DPO pairs => dpo_datasets/resist.json
Saved 13878 train KTO records => dpo_datasets/resist_kto.json
Saved 14835 train rows => dpo_datasets/holistic.csv
Saved 14835 train DPO pairs => dpo_datasets/holistic.json
Saved 29670 train KTO records => dpo_datasets/holistic_kto.json
All done!


In [4]:
# ------------------------------------------------------------------------------
# 4) Use data/qn_id_split_50_50_increments.json to see if train or test
# ------------------------------------------------------------------------------

QNSPLIT_INCR_JSON = "data/qn_id_split_50_50_increments_llama31_8b.json"  # new file

with open(QNSPLIT_INCR_JSON, "r", encoding="utf-8") as f:
    split_data = json.load(f)

# Each key is e.g. train_ids_10, train_ids_20, ..., train_ids_50, plus test_ids
# We'll store them in sets:
train_increments = {
    10: set(split_data["train_ids_10"]),
    20: set(split_data["train_ids_20"]),
    30: set(split_data["train_ids_30"]),
    40: set(split_data["train_ids_40"]),
    50: set(split_data["train_ids_50"]),
}
test_ids = set(split_data["test_ids"])

def get_split(qn_id):
    # We'll label 'test' if qn_id in test_ids, else 'train' if in the final (50%) train_ids_50,
    # else 'unspecified' if it wasn't found (shouldn't happen if the entire dataset is 50/50).
    # This is optional. You might skip if you only plan to filter by the inc sets.
    if qn_id in test_ids:
        return "test"
    elif qn_id in train_increments[50]:
        return "train"
    else:
        return "unspecified"

df["split"] = df["qn_id"].apply(get_split)

# Print overall distribution
grouped = df.groupby(["sample_subset", "split"]).size()
print("\n### Subset distribution ###")
print(grouped)


# ------------------------------------------------------------------------------
# 5) We'll define the same filter_and_save function,
#    but also show how to loop over increments [10, 20, 30, 40, 50].
# ------------------------------------------------------------------------------
import os
import json
import gzip

def filter_and_save(name, condition, qn_id_set, inc_label=None, use_gzip=False):
    """
    1) subdf = df[condition & (df['qn_id'] in qn_id_set)] => i.e. the portion of that subset
       that belongs in this train increment
    2) Save in CSV, JSON, KTO formats, with some naming pattern
    """
    subdf = df[condition & df["qn_id"].isin(qn_id_set)].copy()
    if subdf.empty:
        print(f"Skipping {name}, no rows found for inc={inc_label}")
        return

    suffix = f"_{inc_label}" if inc_label else ""  # e.g. "_10", "_20"
    # CSV
    csv_ext = ".csv.gz" if use_gzip else ".csv"
    csv_path = os.path.join(OUT_DIR, f"{name}{suffix}{csv_ext}")
    if use_gzip:
        with gzip.open(csv_path, "wt", encoding="utf-8") as f:
            subdf.to_csv(f, index=False)
    else:
        subdf.to_csv(csv_path, index=False)
    print(f"Saved {len(subdf)} rows => {csv_path}")

    # JSON in DPO format
    out_list = []
    for row in subdf.to_dict(orient="records"):
        conv = []
        for turn in row["conversations"]:
            speaker = "human" if turn["role"]=="user" else "gpt"
            conv.append({"from": speaker, "value": turn["content"]})

        item = {
            "conversations": conv,
            "chosen": {"from": "gpt", "value": row["chosen"]},
            "rejected": {"from": "gpt", "value": row["rejected"]},
            "meta": {
                "qn_id": row["qn_id"],
                "split": row["split"],  # e.g. 'test' or 'train'
                "sample_subset": row["sample_subset"],
                "appeal_technique": row["appeal_technique"],
                "appeal_setting": row["appeal_setting"],
                "num_turns": row["num_turns"],
                "source": row["source"],
                "category": row["category"],
                "correct_letter": row["correct_letter"],
                "target_letter": row["target_letter"],
                "initial_model_letter": row["initial_model_letter"]
            }
        }
        out_list.append(item)

    json_ext = ".json.gz" if use_gzip else ".json"
    json_path = os.path.join(OUT_DIR, f"{name}{suffix}{json_ext}")
    if use_gzip:
        with gzip.open(json_path, "wt", encoding="utf-8") as f:
            json.dump(out_list, f, ensure_ascii=False, indent=2)
    else:
        with open(json_path, "w", encoding="utf-8") as f:
            json.dump(out_list, f, ensure_ascii=False, indent=2)
    print(f"Saved {len(out_list)} DPO pairs => {json_path}")

    # KTO version
    kto_out = []
    for row in subdf.to_dict(orient="records"):
        base_conv = []
        for turn in row["conversations"]:
            speaker = "human" if turn["role"]=="user" else "gpt"
            base_conv.append({"from": speaker, "value": turn["content"]})

        chosen_item = {
            "conversations": base_conv,
            "kto_tag": True,
            "meta": {
                "qn_id": row["qn_id"],
                "split": row["split"],
                "sample_subset": row["sample_subset"],
                "appeal_technique": row["appeal_technique"],
                "appeal_setting": row["appeal_setting"],
                "num_turns": row["num_turns"],
                "source": row["source"],
                "category": row["category"],
                "correct_letter": row["correct_letter"],
                "target_letter": row["target_letter"],
                "initial_model_letter": row["initial_model_letter"]
            }
        }
        kto_out.append(chosen_item)

        rejected_item = {
            "conversations": base_conv,
            "kto_tag": False,
            "meta": {
                "qn_id": row["qn_id"],
                "split": row["split"],
                "sample_subset": row["sample_subset"],
                "appeal_technique": row["appeal_technique"],
                "appeal_setting": row["appeal_setting"],
                "num_turns": row["num_turns"],
                "source": row["source"],
                "category": row["category"],
                "correct_letter": row["correct_letter"],
                "target_letter": row["target_letter"],
                "initial_model_letter": row["initial_model_letter"]
            }
        }
        kto_out.append(rejected_item)

    kto_ext = "_kto.json.gz" if use_gzip else "_kto.json"
    kto_json_path = os.path.join(OUT_DIR, f"{name}{suffix}{kto_ext}")
    if use_gzip:
        with gzip.open(kto_json_path, "wt", encoding="utf-8") as f:
            json.dump(kto_out, f, ensure_ascii=False, indent=2)
    else:
        with open(kto_json_path, "w", encoding="utf-8") as f:
            json.dump(kto_out, f, ensure_ascii=False, indent=2)
    print(f"Saved {len(kto_out)} KTO records => {kto_json_path}")






### Subset distribution ###
sample_subset  split
baseline       test      1122
               train     1124
relent_POS     test     22638
               train    22722
resist_NEG     test     24486
               train    24486
dtype: int64


In [8]:
# ------------------------------------------------------------------------------
# Now produce the subsets for inc in [10,20,30,40,50].
# We'll define the same baseline/resist/holistic logic for each increment
# ------------------------------------------------------------------------------
cond_baseline  = (df["sample_subset"]=="baseline")
cond_resist    = cond_baseline | (df["sample_subset"]=="resist_NEG")
cond_holistic  = cond_baseline | (df["sample_subset"]=="resist_NEG") | (df["sample_subset"]=="relent_POS")

for inc in [10,20,30,40,50]:
    train_ids_inc = train_increments[inc]  # set of qn_ids

    filter_and_save("baseline",  cond_baseline,  train_ids_inc, inc_label=str(inc), use_gzip=True)
    filter_and_save("resist",    cond_resist,    train_ids_inc, inc_label=str(inc), use_gzip=True)
    filter_and_save("holistic",  cond_holistic,  train_ids_inc, inc_label=str(inc), use_gzip=True)

print("All done with incremental subsets!")

Saved 238 rows => dpo_datasets/baseline_10.csv.gz
Saved 238 DPO pairs => dpo_datasets/baseline_10.json.gz
Saved 476 KTO records => dpo_datasets/baseline_10_kto.json.gz
Saved 5404 rows => dpo_datasets/resist_10.csv.gz
Saved 5404 DPO pairs => dpo_datasets/resist_10.json.gz
Saved 10808 KTO records => dpo_datasets/resist_10_kto.json.gz
Saved 10234 rows => dpo_datasets/holistic_10.csv.gz
Saved 10234 DPO pairs => dpo_datasets/holistic_10.json.gz
Saved 20468 KTO records => dpo_datasets/holistic_10_kto.json.gz
Saved 474 rows => dpo_datasets/baseline_20.csv.gz
Saved 474 DPO pairs => dpo_datasets/baseline_20.json.gz
Saved 948 KTO records => dpo_datasets/baseline_20_kto.json.gz
Saved 10764 rows => dpo_datasets/resist_20.csv.gz
Saved 10764 DPO pairs => dpo_datasets/resist_20.json.gz
Saved 21528 KTO records => dpo_datasets/resist_20_kto.json.gz
Saved 20382 rows => dpo_datasets/holistic_20.csv.gz
Saved 20382 DPO pairs => dpo_datasets/holistic_20.json.gz
Saved 40764 KTO records => dpo_datasets/holist